In [71]:
import pandas as pd
import numpy as np

from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup


In [94]:
### CONTROL FUNCTIONS ###
def pass_promotion(driver):
    element = driver.find_element(by=By.XPATH, value="/html/body/div[3]/div[2]/div/div/div/i")
    # Click on the element
    element.click()

def check_and_pass_promotion(driver):
    for i in range(5):
        try:
            # Wait for the element to be visible
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f"/html/body/div[3]/div[{i}]/div/div/div/i"))
            )
            # Click on the element
            element.click()
            
        except:
            pass

    

In [89]:
WAIT = WebDriverWait(driver, 10) # 10 seconds timeout
BASE_URL = "https://www.pdfdrive.com"

def start_crawling():
    driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver' )

    # Set up WebDriverWait with a timeout

    driver.get(BASE_URL)

    driver.maximize_window()
    time.sleep(10)
    return driver

In [74]:
# import the book_level.csv file
book_level = pd.read_csv('book_levels.csv')
book_level.head()

,Title,Author,Language Level
0,Hop on Pop,Dr. Seuss,level_6a
1,"Jesse Bear,What Will You Wear?",Nancy White Carlstrom,level_6a
2,Eating the Alphabet: Fruits and Vegetables fro...,Lois Ehlert,level_6a
3,Black? White! Day? Night!,Laura Vaccaro Seeger,level_6a
4,Big Red Barn,Margaret Wise Brown,level_6a


In [76]:
def search_book(driver, book_name):
    check_and_pass_promotion(driver)
    driver.find_element(By.ID, "q").send_keys(book_name)
    driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/button").click()
    print(f'Entering book name {book_name}')

# search_book(driver, book_name)

Entering book name Hop on Pop


In [78]:
def get_the_first_book_hyperlink(html_source):
    # Parse the HTML source with BeautifulSoup
    soup = BeautifulSoup(html_source, 'lxml')

    # Find the 'a' tag with the class 'ai-search'
    a_tag = soup.find('a', class_='ai-search')

    # Extract the hyperlink from the 'a' tag
    hyperlink = a_tag['href']

    print(hyperlink)

    url = BASE_URL + hyperlink

    return url

def click_book_details(driver):
    check_and_pass_promotion(driver)
    # Get the entire HTML source of the current page
    html_source = driver.page_source

    url = get_the_first_book_hyperlink(html_source)
    driver.get(url)

    # Wait for the page's title to be present
    WAIT.until(EC.presence_of_element_located((By.TAG_NAME, 'title')))

    print("Clicked on the book details")
    
    return driver

# driver = click_book_details(driver)


/hop-on-pop-beginner-books-e185002882.html
Clicked on the book details


In [79]:
def get_download_button_hyperlink(html_source):
    """
    Click the download button in the book details page
    """
    soup = BeautifulSoup(html_source, 'lxml')

    # Find the 'a' tag with the id 'download-button-link'
    a_tag = soup.find('a', id='download-button-link')

    # Extract the href attribute from the 'a' tag
    hyperlink = a_tag['href']

    url = BASE_URL + hyperlink

    return url


def click_download_button(driver):
    # Get the entire HTML source of the current page
    html_source = driver.page_source

    url = get_download_button_hyperlink(html_source)
    driver.get(url)

    # Wait for the page's title to be present
    WAIT.until(EC.presence_of_element_located((By.TAG_NAME, 'title')))

    print("Clicked to download button")
    
    return driver

# click_download_button(driver)

Clicked to download button


<selenium.webdriver.chrome.webdriver.WebDriver (session="22aee431923954099d08697a8235cdf9")>

In [88]:
def get_download_link():
    wait = WebDriverWait(driver, 20) # 10 seconds timeout

    # Wait for the button to be visible and clickable
    button = wait.until(EC.element_to_be_clickable((By.XPATH, '//a[@class="btn btn-primary btn-user"]')))

    # Extract the hyperlink from the button
    hyperlink = button.get_attribute('href')
    print(hyperlink)

    return hyperlink

def download_book(driver):
    check_and_pass_promotion(driver)
    wait = WebDriverWait(driver, 20) # 10 seconds timeout

    # Get the entire HTML source of the current page
    html_source = driver.page_source

    url = get_download_link()
    print(url)
    driver.get(url)

    # Wait for the page's title to be present
    WAIT.until(EC.presence_of_element_located((By.TAG_NAME, 'title')))
    
    return driver

# download_book(driver)

https://www.pdfdrive.com/download.pdf?id=185002882&h=65f5400b93728ba2f8f9dc08767b3690&u=cache&ext=pdf
https://www.pdfdrive.com/download.pdf?id=185002882&h=65f5400b93728ba2f8f9dc08767b3690&u=cache&ext=pdf


<selenium.webdriver.chrome.webdriver.WebDriver (session="c1b508e498743c8360345ade985e42dd")>

In [ ]:
def check_book_file_type(driver):
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, 'html.parser')
    text = soup.get_text().strip()

    is_epub = 'EPUB' in text
    return is_epub

In [90]:
# import the book_level.csv file
book_level = pd.read_csv('book_levels.csv')

driver = start_crawling()

for i in range(0, len(book_level)):
    book_name = book_level['Title'][i]
    print(book_name)
    search_book(driver, book_name)
    driver = click_book_details(driver)
    driver = click_download_button(driver)
    driver = download_book(driver)
    time.sleep(10)

/var/folders/6_/lmvrh06d1xd8hmgpjs0_y8qm0000gn/T/ipykernel_65242/3874679089.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver' )


Hop on Pop
Entering book name Hop on Pop
/hop-on-pop-beginner-books-e185002882.html
Clicked on the book details
Clicked to download button
https://www.pdfdrive.com/download.pdf?id=185002882&h=65f5400b93728ba2f8f9dc08767b3690&u=cache&ext=pdf
https://www.pdfdrive.com/download.pdf?id=185002882&h=65f5400b93728ba2f8f9dc08767b3690&u=cache&ext=pdf
Jesse Bear,What Will You Wear?
Entering book name Jesse Bear,What Will You Wear?
/calm-the-fck-down-how-to-control-what-you-can-and-accept-what-you-cant-so-you-can-stop-freaking-e157109618.html
Clicked on the book details
Clicked to download button
https://www.pdfdrive.com/download.pdf?id=157109618&h=6f999ffb718eb996f5d8597b63fe7ecb&u=cache&ext=epub
https://www.pdfdrive.com/download.pdf?id=157109618&h=6f999ffb718eb996f5d8597b63fe7ecb&u=cache&ext=epub
Eating the Alphabet: Fruits and Vegetables from A to Z
Entering book name Eating the Alphabet: Fruits and Vegetables from A to Z
/the-grow-your-own-food-handbook-a-back-to-basics-guide-to-planting-growi

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x0000000107fa0ae8 chromedriver + 4958952
1   chromedriver                        0x0000000107f987b3 chromedriver + 4925363
2   chromedriver                        0x0000000107b5a051 chromedriver + 475217
3   chromedriver                        0x0000000107b95c2c chromedriver + 719916
4   chromedriver                        0x0000000107b950f6 chromedriver + 717046
5   chromedriver                        0x0000000107bc3c12 chromedriver + 908306
6   chromedriver                        0x0000000107b8f8f1 chromedriver + 694513
7   chromedriver                        0x0000000107bc3dce chromedriver + 908750
8   chromedriver                        0x0000000107bdf25b chromedriver + 1020507
9   chromedriver                        0x0000000107bc39e3 chromedriver + 907747
10  chromedriver                        0x0000000107b8daef chromedri

In [93]:
pass_promotion(driver)


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=112.0.5615.137)
Stacktrace:
0   chromedriver                        0x0000000107fa0ae8 chromedriver + 4958952
1   chromedriver                        0x0000000107f987b3 chromedriver + 4925363
2   chromedriver                        0x0000000107b5a051 chromedriver + 475217
3   chromedriver                        0x0000000107b9f2e8 chromedriver + 758504
4   chromedriver                        0x0000000107b90283 chromedriver + 696963
5   chromedriver                        0x0000000107bc3c12 chromedriver + 908306
6   chromedriver                        0x0000000107b8f8f1 chromedriver + 694513
7   chromedriver                        0x0000000107bc3dce chromedriver + 908750
8   chromedriver                        0x0000000107bdf25b chromedriver + 1020507
9   chromedriver                        0x0000000107bc39e3 chromedriver + 907747
10  chromedriver                        0x0000000107b8daef chromedri

In [82]:
search_book(driver, book_name)

Entering book name Hop on Pop


In [ ]:
<input type="text" maxlength="200" name="q" id="q" autocomplete="off" 
placeholder="Search for books, tutorials, manuals and authors in 84,037,565 files" value="" required="">